In [1]:
import pandas as pd
import numpy as np
import json
import requests

In [2]:
games = pd.read_csv(
        '/Users/mattbubb/Spring_2021/DS_340W/Sports-Betting-340W/Data/nba_games_all.csv')
#colToDrop = ['oreb', 'dreb', 'reb', 'ast',
                 #'stl', 'blk', 'tov', 'pf', 'pts', 'min', 'fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a', 'fg3_pct']
g = games
moneyLine = pd.read_csv(
        '/Users/mattbubb/Spring_2021/DS_340W/Sports-Betting-340W/Data/nba_betting_money_line.csv')


In [3]:
moneyLine = pd.read_csv(
        '/Users/mattbubb/Spring_2021/DS_340W/Sports-Betting-340W/Data/nba_betting_money_line.csv')
g = g.sort_values('game_date', ascending=False).merge(moneyLine, how='right', on='game_id').dropna()


In [39]:
# TAKES ABOUT 5 MIN TO RUN

rowCount = 0
gameCount = 0
g['joinOn'] = 0
for i in range(g.shape[0]):
    if rowCount < 2:
        g.loc[i,'joinOn'] = gameCount
        rowCount += 1
    else:
        gameCount += 1
        g.loc[i,'joinOn'] = gameCount
        rowCount = 1
g.dropna()

,game_id,game_date,matchup,team_id_x,is_home,wl,w,l,w_pct,min,...,season_year,season_type,season,book_name,book_id,team_id_y,a_team_id_y,price1,price2,joinOn
0,41100314.0,2012-06-02,OKC vs. SAS,1.610613e+09,f,L,10.0,2.0,0.833,240.0,...,2011.0,Playoffs,2011-12,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,165.0,-183.0,0.0
1,41100314.0,2012-06-02,SAS @ OKC,1.610613e+09,t,W,10.0,3.0,0.769,240.0,...,2011.0,Playoffs,2011-12,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,165.0,-183.0,0.0
2,41100314.0,2012-06-02,OKC vs. SAS,1.610613e+09,f,L,10.0,2.0,0.833,240.0,...,2011.0,Playoffs,2011-12,5Dimes,19.0,1.610613e+09,1.610613e+09,165.0,-175.0,1.0
3,41100314.0,2012-06-02,SAS @ OKC,1.610613e+09,t,W,10.0,3.0,0.769,240.0,...,2011.0,Playoffs,2011-12,5Dimes,19.0,1.610613e+09,1.610613e+09,165.0,-175.0,1.0
4,41100314.0,2012-06-02,OKC vs. SAS,1.610613e+09,f,L,10.0,2.0,0.833,240.0,...,2011.0,Playoffs,2011-12,Bookmaker,93.0,1.610613e+09,1.610613e+09,160.0,-190.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250568,41700215.0,2018-05-09,PHI @ BOS,1.610613e+09,t,W,4.0,1.0,0.800,240.0,...,2017.0,Playoffs,2017-18,JustBet,1275.0,1.610613e+09,1.610613e+09,100.0,-120.0,125284.0
250569,41700215.0,2018-05-09,BOS vs. PHI,1.610613e+09,f,L,1.0,4.0,0.200,240.0,...,2017.0,Playoffs,2017-18,JustBet,1275.0,1.610613e+09,1.610613e+09,100.0,-120.0,125284.0
250570,41700215.0,2018-05-09,PHI @ BOS,1.610613e+09,t,W,4.0,1.0,0.800,240.0,...,2017.0,Playoffs,2017-18,Sportsbetting,999991.0,1.610613e+09,1.610613e+09,105.0,-125.0,125285.0
250571,41700215.0,2018-05-09,BOS vs. PHI,1.610613e+09,f,L,1.0,4.0,0.200,240.0,...,2017.0,Playoffs,2017-18,Sportsbetting,999991.0,1.610613e+09,1.610613e+09,105.0,-125.0,125285.0


In [43]:
#Make sure that every game is correctly identified by last column
droppedNA = g.dropna()
droppedNA.head(10)

,game_id,game_date,matchup,team_id_x,is_home,wl,w,l,w_pct,min,...,season_year,season_type,season,book_name,book_id,team_id_y,a_team_id_y,price1,price2,joinOn
0,41100314.0,2012-06-02,OKC vs. SAS,1.610613e+09,f,L,10.0,2.0,0.833,240.0,...,2011.0,Playoffs,2011-12,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,165.0,-183.0,0.0
1,41100314.0,2012-06-02,SAS @ OKC,1.610613e+09,t,W,10.0,3.0,0.769,240.0,...,2011.0,Playoffs,2011-12,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,165.0,-183.0,0.0
2,41100314.0,2012-06-02,OKC vs. SAS,1.610613e+09,f,L,10.0,2.0,0.833,240.0,...,2011.0,Playoffs,2011-12,5Dimes,19.0,1.610613e+09,1.610613e+09,165.0,-175.0,1.0
3,41100314.0,2012-06-02,SAS @ OKC,1.610613e+09,t,W,10.0,3.0,0.769,240.0,...,2011.0,Playoffs,2011-12,5Dimes,19.0,1.610613e+09,1.610613e+09,165.0,-175.0,1.0
4,41100314.0,2012-06-02,OKC vs. SAS,1.610613e+09,f,L,10.0,2.0,0.833,240.0,...,2011.0,Playoffs,2011-12,Bookmaker,93.0,1.610613e+09,1.610613e+09,160.0,-190.0,2.0
5,41100314.0,2012-06-02,SAS @ OKC,1.610613e+09,t,W,10.0,3.0,0.769,240.0,...,2011.0,Playoffs,2011-12,Bookmaker,93.0,1.610613e+09,1.610613e+09,160.0,-190.0,2.0
6,41100314.0,2012-06-02,OKC vs. SAS,1.610613e+09,f,L,10.0,2.0,0.833,240.0,...,2011.0,Playoffs,2011-12,BetOnline,1096.0,1.610613e+09,1.610613e+09,165.0,-190.0,3.0
7,41100314.0,2012-06-02,SAS @ OKC,1.610613e+09,t,W,10.0,3.0,0.769,240.0,...,2011.0,Playoffs,2011-12,BetOnline,1096.0,1.610613e+09,1.610613e+09,165.0,-190.0,3.0
8,41100314.0,2012-06-02,OKC vs. SAS,1.610613e+09,f,L,10.0,2.0,0.833,240.0,...,2011.0,Playoffs,2011-12,Bovada,999996.0,1.610613e+09,1.610613e+09,155.0,-175.0,4.0
9,41100314.0,2012-06-02,SAS @ OKC,1.610613e+09,t,W,10.0,3.0,0.769,240.0,...,2011.0,Playoffs,2011-12,Bovada,999996.0,1.610613e+09,1.610613e+09,155.0,-175.0,4.0


In [72]:
# Take every other row starting on row 0
home  = g.iloc[0::2, :]
lst = []
for i in home.columns:
    lst.append("team1_"+i)
home.columns = lst
home = home.dropna().rename(columns = {'team1_joinOn':'joinOn'})
home

,team1_game_id,team1_game_date,team1_matchup,team1_team_id_x,team1_is_home,team1_wl,team1_w,team1_l,team1_w_pct,team1_min,...,team1_season_year,team1_season_type,team1_season,team1_book_name,team1_book_id,team1_team_id_y,team1_a_team_id_y,team1_price1,team1_price2,joinOn
0,41100314.0,2012-06-02,OKC vs. SAS,1.610613e+09,f,L,10.0,2.0,0.833,240.0,...,2011.0,Playoffs,2011-12,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,165.0,-183.0,0.0
2,41100314.0,2012-06-02,OKC vs. SAS,1.610613e+09,f,L,10.0,2.0,0.833,240.0,...,2011.0,Playoffs,2011-12,5Dimes,19.0,1.610613e+09,1.610613e+09,165.0,-175.0,1.0
4,41100314.0,2012-06-02,OKC vs. SAS,1.610613e+09,f,L,10.0,2.0,0.833,240.0,...,2011.0,Playoffs,2011-12,Bookmaker,93.0,1.610613e+09,1.610613e+09,160.0,-190.0,2.0
6,41100314.0,2012-06-02,OKC vs. SAS,1.610613e+09,f,L,10.0,2.0,0.833,240.0,...,2011.0,Playoffs,2011-12,BetOnline,1096.0,1.610613e+09,1.610613e+09,165.0,-190.0,3.0
8,41100314.0,2012-06-02,OKC vs. SAS,1.610613e+09,f,L,10.0,2.0,0.833,240.0,...,2011.0,Playoffs,2011-12,Bovada,999996.0,1.610613e+09,1.610613e+09,155.0,-175.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250564,41700215.0,2018-05-09,PHI @ BOS,1.610613e+09,t,W,4.0,1.0,0.800,240.0,...,2017.0,Playoffs,2017-18,Intertops,180.0,1.610613e+09,1.610613e+09,100.0,-120.0,125282.0
250566,41700215.0,2018-05-09,PHI @ BOS,1.610613e+09,t,W,4.0,1.0,0.800,240.0,...,2017.0,Playoffs,2017-18,YouWager,139.0,1.610613e+09,1.610613e+09,105.0,-125.0,125283.0
250568,41700215.0,2018-05-09,PHI @ BOS,1.610613e+09,t,W,4.0,1.0,0.800,240.0,...,2017.0,Playoffs,2017-18,JustBet,1275.0,1.610613e+09,1.610613e+09,100.0,-120.0,125284.0
250570,41700215.0,2018-05-09,PHI @ BOS,1.610613e+09,t,W,4.0,1.0,0.800,240.0,...,2017.0,Playoffs,2017-18,Sportsbetting,999991.0,1.610613e+09,1.610613e+09,105.0,-125.0,125285.0


In [73]:
# Take every other row starting on row 1
away = g.iloc[1::2, :]
lst = []
for i in away.columns:
    lst.append("team2_"+i)
away.columns = lst
away = away.dropna().rename(columns = {'team2_joinOn':'joinOn'})
away

,team2_game_id,team2_game_date,team2_matchup,team2_team_id_x,team2_is_home,team2_wl,team2_w,team2_l,team2_w_pct,team2_min,...,team2_season_year,team2_season_type,team2_season,team2_book_name,team2_book_id,team2_team_id_y,team2_a_team_id_y,team2_price1,team2_price2,joinOn
1,41100314.0,2012-06-02,SAS @ OKC,1.610613e+09,t,W,10.0,3.0,0.769,240.0,...,2011.0,Playoffs,2011-12,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,165.0,-183.0,0.0
3,41100314.0,2012-06-02,SAS @ OKC,1.610613e+09,t,W,10.0,3.0,0.769,240.0,...,2011.0,Playoffs,2011-12,5Dimes,19.0,1.610613e+09,1.610613e+09,165.0,-175.0,1.0
5,41100314.0,2012-06-02,SAS @ OKC,1.610613e+09,t,W,10.0,3.0,0.769,240.0,...,2011.0,Playoffs,2011-12,Bookmaker,93.0,1.610613e+09,1.610613e+09,160.0,-190.0,2.0
7,41100314.0,2012-06-02,SAS @ OKC,1.610613e+09,t,W,10.0,3.0,0.769,240.0,...,2011.0,Playoffs,2011-12,BetOnline,1096.0,1.610613e+09,1.610613e+09,165.0,-190.0,3.0
9,41100314.0,2012-06-02,SAS @ OKC,1.610613e+09,t,W,10.0,3.0,0.769,240.0,...,2011.0,Playoffs,2011-12,Bovada,999996.0,1.610613e+09,1.610613e+09,155.0,-175.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250563,41700215.0,2018-05-09,BOS vs. PHI,1.610613e+09,f,L,1.0,4.0,0.200,240.0,...,2017.0,Playoffs,2017-18,Heritage,169.0,1.610613e+09,1.610613e+09,105.0,-125.0,125281.0
250565,41700215.0,2018-05-09,BOS vs. PHI,1.610613e+09,f,L,1.0,4.0,0.200,240.0,...,2017.0,Playoffs,2017-18,Intertops,180.0,1.610613e+09,1.610613e+09,100.0,-120.0,125282.0
250567,41700215.0,2018-05-09,BOS vs. PHI,1.610613e+09,f,L,1.0,4.0,0.200,240.0,...,2017.0,Playoffs,2017-18,YouWager,139.0,1.610613e+09,1.610613e+09,105.0,-125.0,125283.0
250569,41700215.0,2018-05-09,BOS vs. PHI,1.610613e+09,f,L,1.0,4.0,0.200,240.0,...,2017.0,Playoffs,2017-18,JustBet,1275.0,1.610613e+09,1.610613e+09,100.0,-120.0,125284.0


In [74]:
#How many total games 
masterDf = home.merge(away, how = 'left', on = 'joinOn')
len(np.unique(masterDf['team1_game_id']))

14896

In [75]:
predictDf = masterDf.drop_duplicates('team1_game_id')
predictDf

,team1_game_id,team1_game_date,team1_matchup,team1_team_id_x,team1_is_home,team1_wl,team1_w,team1_l,team1_w_pct,team1_min,...,team2_a_team_id_x,team2_season_year,team2_season_type,team2_season,team2_book_name,team2_book_id,team2_team_id_y,team2_a_team_id_y,team2_price1,team2_price2
0,41100314.0,2012-06-02,OKC vs. SAS,1.610613e+09,f,L,10.0,2.0,0.833,240.0,...,1.610613e+09,2011.0,Playoffs,2011-12,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,165.0,-183.0
10,41100304.0,2012-06-03,MIA @ BOS,1.610613e+09,t,W,10.0,7.0,0.588,265.0,...,1.610613e+09,2011.0,Playoffs,2011-12,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,-128.0,116.0
19,41100315.0,2012-06-04,OKC @ SAS,1.610613e+09,t,L,10.0,3.0,0.769,240.0,...,1.610613e+09,2011.0,Playoffs,2011-12,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,188.0,-210.0
29,41100307.0,2012-06-09,MIA vs. BOS,1.610613e+09,f,L,11.0,9.0,0.550,240.0,...,1.610613e+09,2011.0,Playoffs,2011-12,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,367.0,-427.0
39,21200015.0,2012-11-02,CHA vs. IND,1.610613e+09,f,L,1.0,1.0,0.500,240.0,...,1.610613e+09,2012.0,Regular Season,2012-13,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,-275.0,244.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125197,41700233.0,2018-05-04,GSW @ NOP,1.610613e+09,t,W,1.0,2.0,0.333,240.0,...,1.610613e+09,2017.0,Playoffs,2017-18,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,-210.0,188.0
125207,41700213.0,2018-05-05,BOS @ PHI,1.610613e+09,t,L,0.0,3.0,0.000,265.0,...,1.610613e+09,2017.0,Playoffs,2017-18,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,450.0,-535.0
125217,41700234.0,2018-05-06,GSW @ NOP,1.610613e+09,t,L,1.0,3.0,0.250,240.0,...,1.610613e+09,2017.0,Playoffs,2017-18,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,-248.0,221.0
125227,41700215.0,2018-05-09,PHI @ BOS,1.610613e+09,t,W,4.0,1.0,0.800,240.0,...,1.610613e+09,2017.0,Playoffs,2017-18,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,103.0,-113.0


In [ ]:
# To do 

# need to clean up columns, column names
# Figure out 